In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("cleaned_merged_data5.csv", low_memory=False)

In [ ]:
df.columns

In [ ]:
df[['ssl','GroupName', 'ReviewCycle','elapsed_workdays']].head(5)


In [ ]:
uid = df.groupby("ssl").size()

In [ ]:
#Max number of review cycles for a ssl
maxRC = df.groupby(["ssl"],  sort= False)['ReviewCycle'].max()
maxRC = maxRC.reset_index()

In [ ]:
maxRC.head()
maxRC.shape

In [ ]:
#total time taken per ssl
time = df.groupby(["ssl"],  sort= False)['elapsed_workdays'].aggregate(np.sum)
time = time.reset_index()

In [ ]:
time.shape

In [ ]:
time.head()

In [ ]:
#make elapsed workdays dataframe combining same depts with varying reviewCycle
EW = df.groupby(['ssl','GroupName' , 'Ward'], sort= False)['elapsed_workdays'].aggregate(np.sum)
EW = EW.reset_index()

In [ ]:
EW.shape
EW.head()

In [ ]:
#create agencies dummies 
agencies = pd.get_dummies(EW.GroupName)

In [ ]:
EW = pd.concat([EW, agencies], axis=1)

In [ ]:
EW.columns

In [ ]:
EW.head()

In [ ]:
#groupby ssl, so we have just unique ssls, and the dummies indicate all the agencies involved for a permit
dat = EW.groupby(['ssl', 'Ward'], sort= False)['CFA Review',
       'Chinatown Review', 'DC Water Review', 'DDOT Review', 'DOEE',
       'DOH Review', 'EISF Review', 'Electrical Review', 'Elevator Review',
       'Energy Review', 'Fire Review', 'Green Review', 'HPRB Review',
       'Mechanical Review', 'NCPC Review', 'Plumbing Review',
       'Structural Review', 'WMATA Review', 'White House Review',
       'Zoning Review'].aggregate(np.sum)

dat = dat.reset_index()

In [ ]:
print(dat.shape)
dat.head()

In [ ]:
#factor wards

wards = pd.get_dummies(dat.Ward)
#wards = wards[0:6]
wards.shape

In [ ]:
dat = pd.concat([dat,wards], axis=1 )
dat.shape

In [ ]:
#merge the new DV dataframe with max reviewCycle db and total time taken for a permit db
dat = pd.merge(DV, maxRC, on='ssl', how='outer')
dat = pd.merge(dat, time, on='ssl', how='outer')
#dat = dat.drop(['Ward'], axis=1)

In [ ]:
#start the modeling process: 
#running OLS
print(dat.shape)
dat.head()

In [ ]:
#feature and target
y = dat[['elapsed_workdays']]
X = dat.drop(['ssl', 'elapsed_workdays'], axis=1)


In [ ]:
#y.groupby("elapsed_workdays").size()

In [ ]:

dat.to_csv(r'C:\Users\tneha\Desktop\coursework\anly\project\data\ols.csv')

In [ ]:
import matplotlib.pyplot as plt   #Data visualisation libraries 
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import scale

In [ ]:
X['8.0']

In [ ]:
#split the dataset into test and trains sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
#fit a linear regression model
lm = LinearRegression()
reg = lm.fit(X_train,y_train)
reg

In [ ]:
#view coefficients
for idx, col_name in enumerate(X_train.columns):
    print("The coefficient for {} is {}".format(col_name, lm.coef_[idx]))

In [ ]:
intercept = lm.intercept_
print("The intercept for our model is {}".format(intercept))

In [ ]:
predictions = lm.predict(X_test)
#reg.score(X, y)

In [ ]:
lm.score(X_test, y_test)

In [ ]:
plt.scatter(y_test,predictions)